In [1]:
BEST_MODEL_KEY="best_model"
MODEL_PATH_KEY="model"


In [2]:
import os
os.getcwd()
os.chdir('../')

In [3]:
from collections import namedtuple
ModelEvaluationArtifact=namedtuple("ModelEvaluationArtifact",['evaluated_model_path', 'is_model_accepted'])

In [4]:
from insurence_premium.entity.model_factory import evaluate_regression_model


In [5]:
os.getcwd()

'c:\\Users\\somit\\Downloads\\project_ineuron\\insurence_premium_prediction'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataModelEvaluation:
    root_dir: Path
    model_evaluation_file_path:Path
    time_stamp: str


In [14]:
from insurence_premium.constant import *
from insurence_premium.util.common import (
    
    create_directories,read_yaml,
    load_numpy_array_data,load_data,load_object,write_yaml_file
)
from pathlib import Path
from insurence_premium.entity import (
    DataIngestionConfig,
    DataValidationConfig,
    DataTransformationConfig,
    DataModelTrainerConfig
)
from insurence_premium import logger


class ConfigurationManager:
    def __init__(
        self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH
    ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.time_stamp=CURRENT_TIME_STAMP

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(
        self,
    ) -> DataIngestionConfig:
        ingestion_config = self.config.data_ingestion

        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(ingestion_config.root_dir),
            raw_data=Path(ingestion_config.raw_data),
            ingested_train_dir=Path(ingestion_config.ingested_train_dir),
            ingested_test_dir=Path(ingestion_config.ingested_test_dir),
        )
        return data_ingestion_config

    def get_data_validation_config(self) -> DataValidationConfig:
        validation_config = self.config.data_validation

        data_validation_config = DataValidationConfig(
            root_dir=Path(validation_config.root_dir),
            schema_file_path=Path(validation_config.schema_file_path),
            report_file_path=Path(validation_config.report_file_path),
            report_page_file_path=Path(validation_config.report_page_file_path),
        )
        return data_validation_config

    def get_data_transformation_config(self) -> DataTransformationConfig:
        try:
            transformation_config = self.config.data_transformation
            data_transformation_config = DataTransformationConfig(
                root_dir=Path(transformation_config.root_dir),
                transformed_train_file_path=Path(
                    transformation_config.transformed_train_file_path
                ),
                transformed_test_file_path=Path(
                    transformation_config.transformed_test_file_path
                ),
                preprocessing_dir=Path(transformation_config.preprocessing_dir),
                preprocessing_file_path=Path(
                    transformation_config.preprocessing_file_path
                ),
            )
            return data_transformation_config
            logging.info(f"return: [{data_ingestion_config}]")
        except Exception as e:
            raise e

    def get_model_trainer_config(self) -> DataModelTrainerConfig:
        try:
            model_trainer_config = self.config.data_model_trainer

            data_model_trainer_config = DataModelTrainerConfig(
                root_dir=Path(model_trainer_config.root_dir),
                trained_model_file_path=
                    model_trainer_config.trained_model_file_path
                ,
                base_accuracy=model_trainer_config.base_accuracy,
                model_config_file_path=Path(
                    model_trainer_config.model_config_file_path
                ),
            )
            return data_model_trainer_config
            logger.info(f"model trainer config : {data_model_trainer_config}")
        except Exception as e:
            raise e

    def get_model_evaluation_config(self):
        try:
            data_evaluation_config=self.config.data_model_evaluation
            model_evaluation_file_path=os.path.join(data_evaluation_config.root_dir ,data_evaluation_config.model_evaluation_file_name)
            model_evaluation_config=DataModelEvaluation(
                root_dir= Path(data_evaluation_config.root_dir),
                model_evaluation_file_path= Path(model_evaluation_file_path),
                time_stamp= self.time_stamp

                
            )
            return model_evaluation_config
        except Exception as e:
            raise e

In [16]:
ConfigurationManager().get_model_trainer_config()

[2023-04-04 14:05:09,715: INFO: common]: yaml file: config\config.yaml loaded successfully
[2023-04-04 14:05:09,719: INFO: common]: yaml file: params.yaml loaded successfully
[2023-04-04 14:05:09,724: INFO: common]: created directory at: artifacts


DataModelTrainerConfig(root_dir=WindowsPath('artifacts/data_model_trainer'), trained_model_file_path='artifacts/data_model_trainer/trained_model/model.pkl', base_accuracy=0.6, model_config_file_path=WindowsPath('config/model.yaml'))

In [17]:
from insurence_premium.constant import *

CURRENT_TIME_STAMP

'2023-04-04-14-02-29'

In [10]:
import yaml

In [12]:
read_yaml(CONFIG_FILE_PATH).data_model_trainer

[2023-04-04 14:03:08,609: INFO: common]: yaml file: config\config.yaml loaded successfully


ConfigBox({'root_dir': 'artifacts/data_model_trainer', 'trained_model_file_path': 'artifacts/data_model_trainer/trained_model/model.pkl', 'base_accuracy': 0.6, 'model_config_file_path': 'config/model.yaml'})

In [20]:
ConfigurationManager().get_model_trainer_config()

[2023-04-04 14:05:32,863: INFO: common]: yaml file: config\config.yaml loaded successfully
[2023-04-04 14:05:32,866: INFO: common]: yaml file: params.yaml loaded successfully
[2023-04-04 14:05:32,869: INFO: common]: created directory at: artifacts


DataModelTrainerConfig(root_dir=WindowsPath('artifacts/data_model_trainer'), trained_model_file_path='artifacts/data_model_trainer/trained_model/model.pkl', base_accuracy=0.6, model_config_file_path=WindowsPath('config/model.yaml'))

In [21]:
class ModelEvalution:
    def __init__(self,config:ConfigurationManager() ,model_trainer_artifact:ModelTrainerArtifact):
        self.config=config
        self.model_evaluation_config=self.config.get_model_evaluation_config()
        self.model_trainer_config= self.config.get_model_trainer_config()
        self.model_validation=self.config.get_data_validation_config()
        self.model_ingestion=self.config.get_data_ingestion_config()
        self.model_trainer_artifact=model_trainer_artifact
    def get_best_model(self):
        try:
            
            model = None
            model_evaluation_file_path = self.model_evaluation_config.model_evaluation_file_path

            if not os.path.exists(model_evaluation_file_path):
                write_yaml_file(file_path=model_evaluation_file_path,
                                )
                return model
            model_eval_file_content=read_yaml_file(file_path=model_evaluation_file_path)
            logger.info(f"model_eval_file read successfully")

            eval_content=dict() if model_eval_file_content is None else model_eval_file_content

            if BEST_MODEL_KEY  not in eval_content:
                return model
            model=load_object(file_path=model_eval_file_content[BEST_MODEL_KEY][MODEL_PATH_KEY]) 
            return model
                
        except Exception as e:
            raise e

    def update_evaluation_report(self, model_evaluation_artifact: ModelEvaluationArtifact):
        try:
            eval_file_path = self.model_evaluation_config.model_evaluation_file_path
            model_eval_content = read_yaml_file(file_path=eval_file_path)
            model_eval_content = dict() if model_eval_content is None else model_eval_content
            
            
            previous_best_model = None
            if BEST_MODEL_KEY in model_eval_content:
                previous_best_model = model_eval_content[BEST_MODEL_KEY]

            logger.info(f"Previous eval result: {model_eval_content}")
            eval_result = {
                BEST_MODEL_KEY: {
                    MODEL_PATH_KEY: model_evaluation_artifact.evaluated_model_path,
                }
            }

            if previous_best_model is not None:
                model_history = {self.model_evaluation_config.time_stamp: previous_best_model}
                if HISTORY_KEY not in model_eval_content:
                    history = {HISTORY_KEY: model_history}
                    eval_result.update(history)
                else:
                    model_eval_content[HISTORY_KEY].update(model_history)

            model_eval_content.update(eval_result)
            logger.info(f"Updated eval result:{model_eval_content}")
            write_yaml_file(file_path=eval_file_path, data=model_eval_content)
        except Exception as e:
            raise e    


    def initiate_model_evaluation(self) -> ModelEvaluationArtifact:
        try:
            trained_model_file_path = self.model_trainer_config.trained_model_file_path
            trained_model_object = load_object(file_path=trained_model_file_path)

            train_file_path = self.model_ingestion.ingested_train_dir
            test_file_path = self.model_ingestion.ingested_test_dir

            schema_file_path = self.model_validation.schema_file_path

            train_dataframe = load_data(file_path=train_file_path,
                                                           schema_file_path=schema_file_path,
                                                           )
            test_dataframe = load_data(file_path=test_file_path,
                                                          schema_file_path=schema_file_path,
                                                          )
            schema_content = read_yaml(path_to_yaml=schema_file_path)
            target_column_name = schema_content[TARGET_COLUMN_KEY]

            # target_column
            logger.info(f"Converting target column into numpy array.")
            train_target_arr = np.array(train_dataframe[target_column_name])
            test_target_arr = np.array(test_dataframe[target_column_name])
            logger.info(f"Conversion completed target column into numpy array.")

            # dropping target column from the dataframe
            logger.info(f"Dropping target column from the dataframe.")
            train_dataframe.drop(target_column_name, axis=1, inplace=True)
            test_dataframe.drop(target_column_name, axis=1, inplace=True)
            logger.info(f"Dropping target column from the dataframe completed.")

            model = self.get_best_model()

            if model is None:
                logger.info("Not found any existing model. Hence accepting trained model")
                model_evaluation_artifact = ModelEvaluationArtifact(evaluated_model_path=trained_model_file_path,
                                                                    is_model_accepted=True)
                self.update_evaluation_report(model_evaluation_artifact)
                logger.info(f"Model accepted. Model eval artifact {model_evaluation_artifact} created")
                return model_evaluation_artifact

            model_list = [model, trained_model_object]

            metric_info_artifact = evaluate_regression_model(model_list=model_list,
                                                               X_train=train_dataframe,
                                                               y_train=train_target_arr,
                                                               X_test=test_dataframe,
                                                               y_test=test_target_arr,
                                                               base_accuracy=self.model_trainer_artifact.model_accuracy,
                                                               )
            logger.info(f"Model evaluation completed. model metric artifact: {metric_info_artifact}")

            if metric_info_artifact is None:
                response = ModelEvaluationArtifact(is_model_accepted=False,
                                                   evaluated_model_path=trained_model_file_path
                                                   )
                logger.info(response)
                return response

            if metric_info_artifact.index_number == 1:
                model_evaluation_artifact = ModelEvaluationArtifact(evaluated_model_path=trained_model_file_path,
                                                                    is_model_accepted=True)
                self.update_evaluation_report(model_evaluation_artifact)
                logger.info(f"Model accepted. Model eval artifact {model_evaluation_artifact} created")

            else:
                logger.info("Trained model is no better than existing model hence not accepting trained model")
                model_evaluation_artifact = ModelEvaluationArtifact(evaluated_model_path=trained_model_file_path,
                                                                    is_model_accepted=False)
            return model_evaluation_artifact
        except Exception as e:
            raise e

    def __del__(self):
        logging.info(f"{'=' * 20}Model Evaluation log completed.{'=' * 20} ")

[2023-04-04 14:05:36,103: INFO: common]: yaml file: config\config.yaml loaded successfully
[2023-04-04 14:05:36,110: INFO: common]: yaml file: params.yaml loaded successfully
[2023-04-04 14:05:36,113: INFO: common]: created directory at: artifacts


In [19]:
from insurence_premium.entity.model_entity import ModelTrainerArtifact
import numpy as np

In [25]:
try:
    con=ConfigurationManager()
    model_evaluation=ModelEvalution(config=con ,model_trainer_artifact=ModelTrainerArtifact)
    model_evaluation.initiate_model_evaluation()
except Exception as e:
    raise e

[2023-04-04 14:06:48,901: INFO: common]: yaml file: config\config.yaml loaded successfully
[2023-04-04 14:06:48,909: INFO: common]: yaml file: params.yaml loaded successfully
[2023-04-04 14:06:48,911: INFO: common]: created directory at: artifacts
[2023-04-04 14:06:48,928: INFO: common]: yaml file: config\schema.yaml loaded successfully
[2023-04-04 14:06:48,947: INFO: common]: yaml file: config\schema.yaml loaded successfully
[2023-04-04 14:06:48,995: INFO: common]: yaml file: config\schema.yaml loaded successfully
[2023-04-04 14:06:48,998: INFO: 2686975707]: Converting target column into numpy array.
[2023-04-04 14:06:49,002: INFO: 2686975707]: Conversion completed target column into numpy array.
[2023-04-04 14:06:49,005: INFO: 2686975707]: Dropping target column from the dataframe.
[2023-04-04 14:06:49,010: INFO: 2686975707]: Dropping target column from the dataframe completed.
[2023-04-04 14:06:49,013: INFO: 2686975707]: Not found any existing model. Hence accepting trained model
[2

In [23]:
def read_yaml_file(file_path:str):
    try:
        with open(file_path ,'r') as f:
            response =yaml.safe_load(f)
            return response

    except Exception as e:
        raise e